# Introduction

What we've introduced in chapter 5~8, are all value-based methods, among which Q-learning deals with finite states, DQN and improved DQN can be used for descrete states. In RL, desipte value-based methods, there exists another powerful branch: policy-based methods. The difference is that, value-base dmethods learn the value functions and then lead to a policy accordingly, during which there does not exist a explicit policy, while policy-based methods directly learn a policy explicitly. Policy Gradient (PG) is a method belongs to policy-based methods. 

# Policy Gradient

In PG and other policy-based methods, the idea is to parameterize the policy. Assume the target policy $\pi_\theta$ is a stochastic policy and is differentiable everywhere in its domain, among which $\theta$ is the set of all the parameters of the policy. We can use a linear or neural network to model the policy by outputing a distribution of actions given a state as input. The objective of the training is to find the optimal policy and maximize the expected return of this optimal policy in the environment: $$J(\theta) = \mathbb{E}_{s_0}[V^{\pi_\theta}(s_0)], $$ where $s_0$ represents the inital state. 

Now that we have the objective, we can differentiate $J(\theta)$ and adopt gradent ascent to maximize $J(\theta)$ for optimality. 

$$\nabla_\theta V^{\pi_\theta}(s) = \nabla_\theta (\sum_{a \in \mathcal{A}} \pi_\theta(a|s) Q^{\pi_\theta}(s, a)) \\ \quad = \sum_{a \in \mathcal{A}} (\nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a) +\pi_\theta(a|s)  \nabla_\theta Q^{\pi_\theta}(s, a)) \\ \quad = \sum_{a \in \mathcal{A}} (\nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a) +\pi_\theta(a|s)  \nabla_\theta \sum_{s', r}p(s', r|s, a)(r + \gamma V^{\pi_\theta}(s'))) \\ \quad = \sum_{a \in \mathcal{A}} (\nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a) +\gamma \pi_\theta(a|s)  \nabla_\theta \sum_{s', r}p(s', r|s, a)V^{\pi_\theta}(s')) \\ \quad = \sum_{a \in \mathcal{A}} (\nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a) +\gamma \pi_\theta(a|s)  \nabla_\theta \sum_{s'}p(s'|s, a)V^{\pi_\theta}(s'))$$

For simplification, let $\phi(s) = \sum_{a \in \mathcal{A}} \nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a)$, we have:
$$\nabla_\theta V^{\pi_\theta}(s) = \phi(s) +  \gamma \sum_{a} \pi_\theta(a|s)  \nabla_\theta \sum_{s'}p(s'|s, a)V^{\pi_\theta}(s') \\ \quad = \phi(s) +  \gamma \sum_{a} \sum_{s'} \pi_\theta(a|s) p(s'|s, a) \nabla_\theta V^{\pi_\theta}(s') $$

Define $d^{\pi_\theta}(s \rightarrow k, x)$ be the probability of policy $\pi_\theta$ start from state $s$ to state $x$ after $k$ steps, we have:
$$\nabla_\theta V^{\pi_\theta}(s) = \phi(s) +  \gamma \sum_{a} \sum_{s'} d^{\pi_\theta}(s \rightarrow 1, s') \nabla_\theta V^{\pi_\theta}(s') \\ \quad = \phi(s)+  \gamma \sum_{a} \sum_{s'} d^{\pi_\theta}(s \rightarrow 1, s') [\phi(s') +  \gamma \sum_{a'} \sum_{s''} d^{\pi_\theta}(s' \rightarrow 1, s'') \nabla_\theta V^{\pi_\theta}(s'') ] \\ \quad = \phi(s) +  \gamma \sum_{s'} d^{\pi_\theta}(s \rightarrow 1, s') \phi(s') +  \gamma^2 \sum_{s} d^{\pi_\theta}(s \rightarrow 2, s'') \nabla_\theta V^{\pi_\theta}(s'') \\ \quad = \phi(s) +  \gamma \sum_{s'} d^{\pi_\theta}(s \rightarrow 1, s')\phi(s') +  \gamma^2 \sum_{s''} d^{\pi_\theta}(s \rightarrow 2, s'') \phi(s'') + \gamma^3 \sum_{s'''} d^{\pi_\theta}(s \rightarrow 3, s''') \nabla_\theta V^{\pi_\theta}(s''') \\ \quad = ... \\ \quad = \sum_{x \in \mathcal{S}} \sum_{k=0}^\infty \gamma^k d^{\pi_\theta}(s \rightarrow k, x) \phi(x)$$

Let $\eta(s) = \mathbb{E}_{s_0} [\sum_{k=0}^\infty \gamma^k d^{\pi_\theta}(s_0 \rightarrow k, x)]$ and recall in chapter 3 we have $\nu^{\pi}(s) = (1-\gamma) \sum_{t=0}^\infty \gamma^t P^{\pi}_t(s) = \frac{\eta(s)}{\sum_s \eta(s)}$, we have

$$\nabla_\theta J(\theta) = \mathbb{E}_{s_0}[V^{\pi_\theta}(s_0)] \\ \quad = \mathbb{E}_{s_0}[\sum_{x \in \mathcal{S}} \sum_{k=0}^\infty \gamma^k d^{\pi_\theta}(s_0 \rightarrow k, x) \phi(x)] \\ \quad = \sum_{s} \mathbb{E}_{s_0}[\sum_{k=0}^\infty \gamma^k d^{\pi_\theta}(s_0 \rightarrow k, s)] \phi(s) \\ \quad = \sum_{s} \eta(s)\phi(s) \\ \quad = (\sum_{s} \eta(s)) \sum_{s} \frac{\eta(s)}{\sum_{s}\eta(s)} \phi(s) \\ \propto \sum_{s} \frac{\eta(s)}{\sum_{s}\eta(s)} \phi(s) \\ \quad = \sum_{s} \nu(s) \phi(s) \\ \quad = \sum_{s} \nu(s) \sum_{a} \nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a)$$

Hence, we have: 

$$\nabla_\theta J(\theta) \propto \sum_{s} \nu(s) \sum_{a} \nabla_\theta \pi_\theta(a|s) Q^{\pi_\theta}(s, a) \\ \quad = \sum_{s} \nu(s) \sum_{a} \pi_\theta(a|s)Q^{\pi_\theta}(s, a) \frac{\nabla_\theta \pi_\theta(a|s) }{ \pi_\theta(a|s) } \\ \quad = \mathbb{E}_{\pi_{\theta}}[Q^{\pi_\theta}(s, a) \nabla_\theta \log(\pi_\theta(a|s))]$$


By the inference above, we can see that $\nabla_\theta J(\theta) \propto \mathbb{E}_{\pi_{\theta}}[Q^{\pi_\theta}(s, a) \nabla_\theta \log(\pi_\theta(a|s))]$ can be used to update the gradient. Notice that the expectation is taken with respect to $\mathbb{E}_{\pi_{\theta}}$, so gradient policy algorithm is on-policy, which means you have to use the data sampled based on current policy for the update. 

A direct understanding to this update formula is that, the policy is encouraged to sample data that brings more value. 

In torch, we can easily implement $\nabla_\theta log(\pi_\theta(a|s))$, but what about $Q^{\pi_\theta}(s,a)$? The PG algorithm we introduce below, the REINFORCE algorithm, estimates it by MC. For a finite-step env, REINFORCE gives us $Q^{\pi_\theta}(s,a) = \sum_{t=1}^T (\sum_{t'=t}^T \gamma^{t'-t} r_{t'})$, where $T$ is the maximal steps allowed.

The REINFORCE algorithm can be written as: 
- Init $\theta$
- for episode $e = 1$ to $E$:
    - sample a trajectory $\{s_1, a_1, r_1, s_2, a_2, r_2, ..., s_T, a_T, r_T\}$ under current policy $\pi_\theta$
    - calculate the return of each timestep $\psi_t = \sum_{t'=t}^T \gamma^{t'-t} r_{t'}$
    - $\theta \leftarrow \theta + \alpha \sum_t^T \psi_t \nabla_\theta log(\pi_\theta(a|s))$
- end for

Now, let us implement it.